In [1]:
# Pip install required dependencies
! pip install openai
! pip install pinecone-client[grpc]
! pip install pd 


import os
import csv

import openai
import pinecone 
from kaggle_secrets import UserSecretsClient
import pandas as pd

# Load OpenAI and Pinecone API keys via Kaggle secrets, to avoid leaking secrets via Jupyter notebook
OPENAI_API_KEY = UserSecretsClient().get_secret("OPENAI_API_KEY")
PINECONE_API_KEY = UserSecretsClient().get_secret("PINECONE_API_KEY")

index_name = "the-office-oracle"

pinecone.init(api_key=PINECONE_API_KEY, environment='us-west4-gcp-free')

index = pinecone.Index(index_name)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
google-cloud-artifact-registry 1.8.1 requires google-api-core[grpc]!=2.0.*,!=2.1.*,!=2.10.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,!=2.9.*,<3.0.0de

/opt/conda/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
user_query = "What did Dwight find in the ceiling?"

model_name = "text-embedding-ada-002"

vector = openai.Embedding.create(
    api_key=OPENAI_API_KEY, 
    model=model_name,
    input=user_query
)

query_response = index.query(
    top_k=3,
    include_values=True,
    include_metadata=True,
    vector=vector.data[0].embedding,
)

if query_response is not None: 
    print("Got query_response...")

for match in query_response['matches']: 
    print(f'text: {match.metadata["text"]}')

Got query_response...
text: Dwight spots animal droppings on the carpet. He peeks above the ceiling tiles and initially concludes that there is a bird stuck in the air vents, but is surprised to discover that it is actually a bat. The bat escapes, and everyone panics (except for Jim, Oscar, and a somewhat amused Creed). Kelly and Meredith scream, Kevin runs into the camera, Pam hides near the coat rack, Karen hides under her desk, and Angela falls flat on the floor and prays. Stanley heads home (with a very dry "Good-bye") as Dwight shuts the bat in a conference room. Angela puts on a clear plastic rain bonnet so the bat won't poop on her head.
Jim calls animal control and tells Dwight that it won't be at the office until 6 p.m. As Jim and Dwight talk, Jim tells Dwight that he doesn't feel good. He tells Dwight he might have been bitten by the bat and implies he might be turning into a vampire. Dwight looks on in confusion. Jim and Karen continue pranking Dwight, speaking in slow, deli

In [4]:
! pip install langchain
! pip install sentence_transformers 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=1de8f4fa6d68e20d6d2bc5b51338ce63a200211601749194f5a86f77d21d884b
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [6]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

from langchain.vectorstores import Pinecone
db = Pinecone.from_existing_index(index_name, embeddings)

query = "What happens with Ryan and his desk?"
docs = db.similarity_search(query)

for doc in docs: 
    print(f'Retrieved doc:{doc}')


Retrieved doc:page_content='Ryan redirects the speech into a question-and-answer session. Michael solemnly states that "real business is done on paper," and tells the students to write that fact down. The audience quickly complies by typing it on their laptops.\nThe Q&A session intensifies, with students wondering aloud how a small business could survive against the "five Goliaths" of the paper industry. Michael defends his company, commenting at the students\' inability to understand the situation based upon their age and lack of experience. He also states that the United States faces "five goliaths" as well but only states four: Al-Qaeda, Global Warming, sex predators, and mercury poisoning. When a student uses Ryan\'s prediction of Dunder Mifflin\'s obsolescence as proof of his point, Michael is shocked to hear that Ryan had made such a statement. He informs the room that Ryan still hasn\'t made a sale, started a fire in his attempt to make a cheese pita, and "a lot of people think 

In [7]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

qa = RetrievalQA.from_chain_type(llm=OpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo-16k"), chain_type="stuff", retriever=db.as_retriever())

query = "List some dangerous situations Dwight has caused"
resp = qa.run(query)
print(resp)

/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:189: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:769: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


1. Dwight's actions that could have burned down the building or killed Stanley.
2. Dwight mutilating the CPR dummy, costing the company $3500.
3. Dwight tricking his coworkers into signing an apology sheet by pretending it was an attendance sheet.
4. Dwight nearly suffocating Clark with shrink wrap during a hazing plan.
5. Dwight attempting to ride a bicycle across a tight-rope and needing to be rescued by firefighters.
